In [ ]:
################################################################################################################################
######################################## CODE FOR HPS ALGORITHM AND CHORD KNN CLASSIFIER #######################################
########################################   IMPLEMENTED BY MICHAEL CUEVAS, AARON KELLER,  #######################################
########################################   AND DAVID ZANE FOR EECS 352 AT NORTHWESTERN   #######################################
################################################################################################################################

In [ ]:
# This line is a convenience to import most packages you'll need. You may need to import others (e.g. random and cmath)
import IPython, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa, cmath,math, csv
from IPython.display import Audio
from sklearn.datasets import load_iris
import librosa.display
%matplotlib inline

In [ ]:
""" FUNCTION TO LOAD DESIGNATED AUDIO"""
def load_audio(audio_path):
    """
    Loads an audio file and returns the signal and sample rate. Also displays an audio player to play original sound
    
    Parameters
    ----------
    audio_path: string
        path to audio file to load
    
    Output
    ----------
    chord,sr: tuple
        contains the signal as index 0 and the sample rate as index 1
    """
    chord,sr = librosa.load(audio_path, sr=None)
    Audio(chord, rate=sr)
    return (chord, sr)


In [ ]:
""" PLOTTING FUNCTIONS USED IN HPS ANALYSIS """

def plot_chromogram(signal, sr, n_fft, hop_length):
    """
    Displays a chromogram for a given signal
    
    Parameters
    ----------
    signal: 1D Numpy Array
        contains the original audio signal
    sr: int
        sample rate of the audio signal
    n_fft: int
        number of bins for the short time fourier transform
    hop_length: int
        size of each bin of the fourier transform
    """
    chromagram = librosa.feature.chroma_stft(chord, sr=sr, n_fft=2048, hop_length=1024)
    librosa.display.specshow(chromagram,y_axis='chroma',x_axis='time')
    plt.colorbar()
    plt.title('Chromagram')
    plt.tight_layout()
    plt.show()

def plot_fft(FFT, sr):
    """
    Displays a chromogram for a given signal
    
    Parameters
    ----------
    FFT: 1D Numpy Array
        contains the FFT of an audio signal
    sr: int
        sample rate of the audio signal
    """
    Nf = np.shape(FFT)[0]
    below_nyquist = FFT[0:int(Nf/2)+1]

    plt.figure()
    freq_arr = np.arange(int(Nf/2)+1) * (sr/Nf)
    plt.plot(freq_arr, below_nyquist)
    #plt.xlim([0,2000])
    #print(np.max(fft_chroma_test))
    
def plot_hps(hps, sr, type_sig):
    """
    Plots the harmonic product spectrum
    
    Parameters
    ----------
    hps: 1D Numpy Array
        contains the harmonic product spectrum of a signal
    type_sig: string
        contains the type of hps (either 'guitar' or 'piano')
    sr: int
        sample rate of the original audio signal
    """
    if(type_sig == 'guitar'):
        num_dec = 5
    else:
        num_dec = 3
    
    plt.figure()
    win_len = len(hps)
    freq_arr = np.arange(win_len) * ((sr/num_dec)/win_len)
    plt.xlim([0,500])
    plt.plot(freq_arr, hps)
    

In [ ]:
""" FUNCTIONS USED FOR SIGNAL ANALYSIS """

def magnitude_spectrogram(signal, sr):
    """
    Displays a chromogram for a given signal
    
    Parameters
    ----------
    signal: 1D Numpy Array
        contains the original audio signal
    sr: int
        sample rate of the audio signal
    Output
    ----------
    FFT: 1D Numpy Array
        contains the magnitude spectrogram of the original signal
    """
    FFT = np.abs(np.fft.fft(signal))
    #fft_chroma_test = np.abs(fft_chroma_test)
    Nf = np.shape(FFT)[0]
    FFT = FFT[0:int(Nf/2)+1]
    return FFT

def hps_piano(mag_spec):
    """
    Calculates the harmonic product spectrum for a piano
    
    Parameters
    ----------
    mag_spec: 1D Numpy Array
        contains the magnitude spectrogram of the signal
    Output
    ----------
    hps: 1D Numpy Array
        contains the harmonic product spectrum
    """
    # decimates the audio 3 times
    dec1 = sp.signal.decimate(mag_spec, 1)
    dec2 = sp.signal.decimate(mag_spec, 2)
    dec3 = sp.signal.decimate(mag_spec, 4)
    
    # multiplies the decimated audio together to reduce overtones
    hps = np.zeros(len(dec3))
    for i in range(len(dec3)):
        product = dec1[i] * dec2[i] * dec3[i]
        hps[i] = product
    return hps

def hps_guitar(mag_spec):
    """
    Calculates the harmonic product spectrum for a guitar
    
    Parameters
    ----------
    mag_spec: 1D Numpy Array
        contains the magnitude spectrogram of the signal
    Output
    ----------
    hps: 1D Numpy Array
        contains the harmonic product spectrum
    """
    # decimates the audio signal 5 times to reduce overtones
    dec1 = sp.signal.decimate(mag_spec, 1)
    dec2 = sp.signal.decimate(mag_spec, 2)
    dec3 = sp.signal.decimate(mag_spec, 4)
    dec4 = sp.signal.decimate(mag_spec, 8)
    dec5 = sp.signal.decimate(mag_spec, 16)
    
    # multiplies the decimated audio together to reduce overtones
    hps = np.zeros(len(dec5))
    for i in range(len(dec5)):
        product = dec1[i] * dec2[i] * dec3[i] * dec4[i] * dec5[i]
        hps[i] = product
    return hps



In [ ]:
#fft_chroma_test = np.abs(np.fft.fft(chord))
#fft_chroma_test = np.abs(fft_chroma_test)
#Nf = np.shape(fft_chroma_test)[0]
#fft_chroma_test = fft_chroma_test[0:int(Nf/2)+1]

#plt.figure(1)

#freq_arr = np.arange(int(Nf/2)+1) * (sr/Nf)
#plt.plot(freq_arr,fft_chroma_test)
#plt.xlim([0,2000])
#print(np.max(fft_chroma_test))



In [ ]:
#chord = sp.signal.windows.hann(len(chord)) * chord

# load in audio file and specify sound type ('guitar' or 'piano')
path = 'c.wav'
sig_type = 'piano'
chord, sr = load_audio(path)

# calculate magnitude spectrogram with values up to nyquist frequency and plot
spec = magnitude_spectrogram(chord, sr)
plot_fft(spec, sr)

# use HPS algorithm to reduce overtones
if(sig_type == 'guitar'):
    hps = hps_guitar(spec)
else:
    hps = hps_piano(spec)
    
# plot HPS output
plot_hps(hps, sr, sig_type)

# output information about chord analysis

Nf = np.shape(hps)[0]
indices = np.argsort(hps)
if(sig_type == 'guitar'):
    indices = indices*((sr/16)/Nf)
else:
    indices = indices*((sr/4)/Nf)
length = np.shape(indices)[0]
indexName = ''
for i in range(10):
    if(i == 0):
        indexName = "1st "
    if(i == 1):
        indexName = "2nd "
    if(i == 2):
        indexName = "3rd "
    else:
        indexName = str(i+1) + "th "
    print(indexName + "largest value: " + str(indices[length-1-i]) + "\n")


'''
plt.figure(0)
win_len2 = len(hps1)
freq_arr2 = np.arange(win_len2) * ((sr/2)/win_len2)
plt.plot(freq_arr2, hps1)
#plt.xlim([0,2000])

plt.figure(1)
hps2 = sp.signal.decimate(fft_chord, 8)
win_len2 = len(hps2)
freq_arr2 = np.arange(win_len2) * ((sr/8)/win_len2)
plt.plot(freq_arr2, hps2)
plt.xlim([0,2000])


# Calculation for HPS 

y = np.zeros(len(hps5))
for i in range(len(hps5)):
    product = hps1[i] * hps2[i] * hps3[i] * hps4[i] * hps5[i]
    y[i] = product

# Plot for HPS

plt.figure(0)
win_len2 = len(y)
freq_arr2 = np.arange(win_len2) * ((sr/16)/win_len2)
plt.xlim([0,500])
plt.plot(freq_arr2, y)

fft_chroma_test = np.fft.fft(chord)
fft_chroma_test = np.abs(fft_chroma_test)
Nf = np.shape(fft_chroma_test)[0]
fft_chroma_test = fft_chroma_test[0:int(Nf/2)+1] 


plt.figure(1)
freq_arr = np.arange(int(Nf/2)+1) * (sr/Nf)
plt.plot(freq_arr,fft_chroma_test)
plt.xlim([0,500])
print(np.max(fft_chroma_test))
ind = np.argsort(y)
ind = ind*((sr/16)/win_len2)
leng = np.shape(ind)[0]
print(str(ind[leng-4:leng]))

'''



In [ ]:
# load in audio file and specify sound type ('guitar' or 'piano')
path = 'chords.wav'
sig_type = 'piano'
chord, sr = load_audio(path)
chord = chord[0:400000]
display(Audio(chord, rate=sr))

times = librosa.onset.onset_detect(y=chord, sr=sr, units='samples')

# for each onset (chord) detected
for i in times:
    # calculate magnitude spectrogram with values up to nyquist frequency and plot
    spec = magnitude_spectrogram(chord[i:i+6000], sr)
    plot_fft(spec, sr)

    # use HPS algorithm to reduce overtones
    if(sig_type == 'guitar'):
        hps = hps_guitar(spec)
    else:
        hps = hps_piano(spec)

    # plot HPS output
    plot_hps(hps, sr, sig_type)

    # output information about chord analysis

    Nf = np.shape(hps)[0]
    indices = np.argsort(hps)
    if(sig_type == 'guitar'):
        indices = indices*((sr/16)/Nf)
    else:
        indices = indices*((sr/4)/Nf)
    length = np.shape(indices)[0]
    indexName = ''
    print("\nCHORD AT SAMPLE NUMBER: " + str(i+1) + "\n")
    for j in range(10):
        if(j == 0):
            indexName = "1st "
        if(j == 1):
            indexName = "2nd "
        if(j == 2):
            indexName = "3rd "
        else:
            indexName = str(j+1) + "th "
        print(indexName + "largest value: " + str(indices[length-1-j]) + "\n")


"""
chord,sr = load_audio('chords.wav')
chord = chord[0:400000]
print(len(chord))
graph_chromogram(chord, sr=sr, n_fft=2048, hop_length=1024)
Audio(chord[0:400000], rate=sr)

times = librosa.onset.onset_detect(y=chord, sr=sr, units='samples')
print(times)
plt.plot(chord)


for i in times:
    # Graph Chromogram of initial audio
    '''
    chromagram = librosa.feature.chroma_stft(chord[i:i+4000], sr=sr, n_fft=2048, hop_length=1024)
    librosa.display.specshow(chromagram,y_axis='chroma',x_axis='time')
    plt.figure(i+2)
    plt.colorbar()
    plt.title('Chromagram')
    plt.tight_layout()
    plt.show()
    '''
    
    #calc FFT
    fft_chroma_test = np.fft.fft(chord[i:i+8000])
    fft_chroma_test = np.abs(fft_chroma_test)
    Nf = np.shape(fft_chroma_test)[0]
    fft_chroma_test = fft_chroma_test[0:int(Nf/2)+1] 
    
    #plots FFT
    plt.figure(i)
    freq_arr = np.arange(int(Nf/2)+1) * (sr/Nf)
    plt.plot(freq_arr,fft_chroma_test)
    #plt.xlim([0,2000])
    #print(np.max(fft_chroma_test))
    
    # Decimates the FFT (downsamples)
    hps1 = sp.signal.decimate(fft_chroma_test, 1)
    print(np.shape(hps1))
    hps2 = sp.signal.decimate(fft_chroma_test, 2)
    print(np.shape(hps2))
    hps3 = sp.signal.decimate(fft_chroma_test, 4)
    print(np.shape(hps3))
    #hps4 = sp.signal.decimate(fft_chroma_test, 8)
    #hps5 = sp.signal.decimate(fft_chroma_test, 16)
   #hps6 = sp.signal.decimate(fft_chroma_test, 32)
    
    #calculates HPS
    y = np.zeros(len(hps3))
    for j in range(len(hps3)):
        product = hps1[j] * hps2[j] * hps3[j] #* hps4[j] * hps5[j] * hps6[j]
        y[j] = product

    # Plot for HPS

    plt.figure(i+1)
    win_len2 = len(y)
    freq_arr2 = np.arange(win_len2) * ((sr/4)/win_len2)
    plt.xlim([0,1000])
    plt.plot(freq_arr2, y)
    
    '''
    # Plots original FFT for comparison
    fft_chroma_test = np.fft.fft(chord)
    fft_chroma_test = np.abs(fft_chroma_test)
    Nf = np.shape(fft_chroma_test)[0]
    fft_chroma_test = fft_chroma_test[0:int(Nf/2)+1] 
    plt.figure(1)
    '''
    #plots HPS and finds 4 largest peaks
    freq_arr = np.arange(int(Nf/2)+1) * (sr/Nf)
    plt.plot(freq_arr,fft_chroma_test)
    print(np.max(fft_chroma_test))
    ind = np.argsort(y)
    ind = ind*((sr/4)/win_len2)
    leng = np.shape(ind)[0]
    print(str(ind[leng-7:leng]))
    """
